In [ ]:
!pip install rdkit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 25.9 MB/s eta 0:00:00


#Importing Libraries 

In [ ]:
import os
import pandas as pd
import numpy as np
import copy
import argparse
import rdkit
from rdkit import Chem
from rdkit.ML.Descriptors import MoleculeDescriptors
from rdkit.Chem import AllChem, Draw, DataStructs
from rdkit.Chem import inchi
from rdkit import RDLogger  
RDLogger.DisableLog('rdApp.*') 
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from rdkit.Chem.Draw import rdMolDraw2D
from IPython.display import SVG

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix
from tqdm import tqdm
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE
import warnings
warnings.filterwarnings("ignore")

print(rdkit.__version__)

2022.09.5


#Training Dataset

In [ ]:
# Preprocessed Tox21 dataset with pre-assigned clusters
data_training = pd.read_csv("data_train.csv",index_col=0).reset_index(drop=True)
data_training

,smiles,task1,task2,task3,task4,task5,task6,task7,task8,task9,task10,task11
0,CC(=O)N(C)c1cccc(-c2ccnc3c(C(=O)c4cccs4)cnn23)c1,0,0,0,0,0,0,0,-1,0,0,0
1,COc1cc(N)c(Cl)cc1C(=O)OCCCN1CCCCC1.Cl,0,0,0,0,0,0,0,-1,0,0,0
2,CCCCNc1c(C(=O)OCC)cnc2c1cnn2CC,0,0,0,0,0,0,0,0,0,1,0
3,C#Cc1cccc(Nc2ncnc3cc(OCCOC)c(OCCOC)cc23)c1.Cl,0,0,0,0,0,0,0,-1,0,0,1
4,CC1OC2(CCCCC2Oc2cccc(Cl)c2)N=C1O,0,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
11995,CC(C)(C)NC[C@@H](O)COc1nsnc1N1CCOCC1,0,0,0,0,0,0,0,-1,0,0,0
11996,CCC[C@@]1(CCc2ccccc2)CC(O)=C([C@H](CC)c2cccc(N...,0,0,0,0,0,0,0,-1,0,0,0
11997,N=C(O)c1cnc(C2CC2)[nH]1,0,0,0,-1,0,0,0,0,0,0,0
11998,CN=C=O,0,0,0,0,0,0,-1,0,0,0,0


# Checking imabalanced classes for each output 

In [ ]:
for i in range (1,12):
    print(f"task{i}")
    print("----------------")
    print(data_training[f"task{i}"].value_counts()[[0,1]])
    print("**************************** \n")
    


task1
----------------
0    10977
1      462
Name: task1, dtype: int64
**************************** 

task2
----------------
0    11368
1      316
Name: task2, dtype: int64
**************************** 

task3
----------------
0    11038
1      637
Name: task3, dtype: int64
**************************** 

task4
----------------
0    11390
1       46
Name: task4, dtype: int64
**************************** 

task5
----------------
0    11368
1      122
Name: task5, dtype: int64
**************************** 

task6
----------------
0    10990
1       60
Name: task6, dtype: int64
**************************** 

task7
----------------
0    7689
1     306
Name: task7, dtype: int64
**************************** 

task8
----------------
0    8196
1      48
Name: task8, dtype: int64
**************************** 

task9
----------------
0    8374
1     624
Name: task9, dtype: int64
**************************** 

task10
----------------
0    10686
1     1004
Name: task10, dtype: int64
***************

#### We noticed that :

"Class 0" is always much higher than "class 1" 

# Dividing data to train Valdiation sets 

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(data_training["smiles"],data_training.iloc[:,1:],test_size=0.2,random_state=42)

In [ ]:
y_train["task1"]

9182     0
11091    0
6428     0
288      0
2626     0
        ..
11964    0
5191     0
5390     0
860      0
7270     0
Name: task1, Length: 9600, dtype: int64

# Calculate Morgan fingerprints for  training dataset

In [ ]:
# Initialize variables
fp_length = 1024
fps_train = np.zeros((len(x_train), fp_length))

# Calculate Morgan fingerprints and convert to numpy array
for i, smiles in enumerate(tqdm(x_train)):
    mol = Chem.MolFromSmiles(smiles)
    fp_vec = AllChem.GetMorganFingerprintAsBitVect(mol, radius=3, nBits=fp_length)
    arr = np.zeros((1,))
    DataStructs.ConvertToNumpyArray(fp_vec, arr)
    fps_train[i] = arr

100%|██████████| 9600/9600 [00:03<00:00, 2580.61it/s]


 # Calculate Morgan fingerprints for  validation  dataset

In [ ]:

fp_length = 1024
fps_val = np.zeros((len(x_test), fp_length))

# Calculate Morgan fingerprints and convert to numpy array
for i, smiles in enumerate(tqdm(x_test)):
    mol = Chem.MolFromSmiles(smiles)
    fp_vec = AllChem.GetMorganFingerprintAsBitVect(mol, radius=3, nBits=fp_length)
    arr = np.zeros((1,))
    DataStructs.ConvertToNumpyArray(fp_vec, arr)
    fps_val[i] = arr

100%|██████████| 2400/2400 [00:01<00:00, 1932.02it/s]


In [ ]:
fps_train.shape

(9600, 1024)

In [ ]:
fps_val.shape

(2400, 1024)

In [ ]:
idx = (y_train.values[:, 0] != (-1))
y_train.values[idx, 0]

array([0, 0, 0, ..., 0, 0, 0])

In [ ]:
ind=y_train.values[:, 1] != (-1)
uni=np.unique(y_train.values[ind,1], return_counts=True)
print(uni[1][0])

9103


In [ ]:
sampling_strategy = {0: uni[1][0], 1: uni[1][0]}

#Train model on random split

We can now train a random forest model using the Morgan fingerprints as input. Here we don't use the clustering .

In [ ]:
def train_data_upsample (fps_train,fps_val,y_train):
    classifiers=[]
    Y_hat_all=np.zeros(shape=(fps_val.shape[0],11))
    print(Y_hat_all.shape)
    for i in range(y_train.shape[1]):
        classifiers.append(RandomForestClassifier(n_estimators=100, random_state=1024))
        idx_train = (y_train.values[:, i] != (-1))
        X_train=fps_train[idx_train]
        Y_train=y_train.values[idx_train, i]
        uni=np.unique(Y_train, return_counts=True)
        sampling_strategy = {0: uni[1][0], 1: uni[1][0]}
        smote = SMOTE(sampling_strategy=sampling_strategy, random_state=42)
        X_train,Y_train=smote.fit_resample(X_train,Y_train)
        classifiers[i].fit(X_train, Y_train)
        Y_hat_all[:,i]=(classifiers[i].predict_proba(fps_val)[:,1])
    return classifiers,Y_hat_all

In [ ]:
# def train_data_grid (fps_train,fps_val,y_train):
#     best_params=[]
#     best_roc_auc=[]
#     Y_hat_all=np.zeros(shape=(fps_val.shape[0],11))
#     print(Y_hat_all.shape)
#     scoring = 'roc_auc'
#     param_grid = {'n_estimators': [50, 100, 200],
#                   'max_depth': [2, 5, 10],
#                   'min_samples_split': [2, 5, 10],
#                   'min_samples_leaf': [1, 2, 4], 
#                   "criterion":["gini", "entropy"],
#                   "random_state":[1024], 
#                   "class_weight":[{0:1,1:100},{0:1,1:1000}]}
#     for i in range(y_train.shape[1]):
#         rf=RandomForestClassifier( )
#         grid_search = GridSearchCV(estimator=rf, param_grid=param_grid,scoring=scoring, cv=5)
#         idx_train = (y_train.values[:, i] != (-1))
#         X_train=fps_train[idx_train]
#         Y_train=y_train.values[idx_train, i]
#         grid_search.fit(X_train, Y_train)
#         best_params.append(grid_search.best_params_)

#         best_roc_auc.append(roc_auc_score(y_test, grid_search.predict_proba(X_train)[:,1]))

#         Y_hat_all[:,i]=(grid_search.predict_proba[:,1])
#     return Y_hat_all,best_roc_auc

#Metrics 

Area under the ROC curve (AUC)

In [ ]:
def Avg_AUC_score (y_test,Y_hat_all):
    auc_per_task = []
    for j in range(y_test.shape[1]):
        y_true = y_test.values[:, j]
        # mask out unknown samples
        idx = (y_true != (-1))
        # calculate AUC per task
        auc_per_task.append(roc_auc_score(y_true[idx], Y_hat_all[idx,j]))
    avg_auc = np.mean(auc_per_task)
    print(avg_auc)
    return avg_auc

In [ ]:
classifiers,y_hat_all=train_data_upsample (fps_train,fps_val,y_train)

(2400, 11)


In [ ]:
avg_auc=Avg_AUC_score (y_test,y_hat_all)

0.6805792130529553


# Training over All training data

In [ ]:
x_train=data_training["smiles"]
y_train=data_training.iloc[:,1:]

#Testing Dataset

In [ ]:
data_test = pd.read_csv("smiles_test.csv",index_col=0).reset_index(drop=True)
data_test

,smiles
0,OC(COc1ccc(Cl)cc1)=N[C@H]1CC[C@H](N=C(O)COc2cc...
1,CCCO/N=C(/C)c1cc(C(O)=NC(Cc2cc(F)cc(F)c2)[C@@H...
2,COc1cc(Cl)ccc1Cl
3,COc1cc(C(O)=NCc2ccc(OCCN(C)C)cc2)cc(OC)c1OC
4,CCC(=O)O[C@@]1(C(=O)CCl)[C@@H](C)C[C@H]2[C@@H]...
...,...
5891,N#Cc1cc(NC(=O)C(=O)O)c(Cl)c(NC(=O)C(=O)O)c1.NC...
5892,O=c1cccc2n1C[C@@H]1CNC[C@H]2C1
5893,CSCC[C@H](N=C(O)[C@H](Cc1ccccc1)N=C(O)CN=C(O)C...
5894,CCn1cc2c3c(cc(C(O)=NC(Cc4ccccc4)[C@H](O)C[NH2+...


 # Calculate Morgan fingerprints for  Testing  dataset

In [ ]:
# Initialize variables
fp_length = 1024
fps_test = np.zeros((len(data_test), fp_length))

# Calculate Morgan fingerprints and convert to numpy array
for i, smiles in enumerate(tqdm(data_test['smiles'])):
    mol = Chem.MolFromSmiles(smiles)
    fp_vec = AllChem.GetMorganFingerprintAsBitVect(mol, radius=3, nBits=fp_length)
    arr = np.zeros((1,))
    DataStructs.ConvertToNumpyArray(fp_vec, arr)
    fps_test[i] = arr

100%|██████████| 5896/5896 [00:03<00:00, 1693.96it/s]


 # Calculate Morgan fingerprints for  Training  dataset

In [ ]:
# Initialize variables
fp_length = 1024
fps_train = np.zeros((len(x_train), fp_length))

# Calculate Morgan fingerprints and convert to numpy array
for i, smiles in enumerate(tqdm(x_train)):
    mol = Chem.MolFromSmiles(smiles)
    fp_vec = AllChem.GetMorganFingerprintAsBitVect(mol, radius=3, nBits=fp_length)
    arr = np.zeros((1,))
    DataStructs.ConvertToNumpyArray(fp_vec, arr)
    fps_train[i] = arr

100%|██████████| 12000/12000 [00:05<00:00, 2183.67it/s]


#Training the model using Random forest 

In [ ]:
classifiers,y_hat_all=train_data_upsample (fps_train,fps_test,y_train)

(5896, 11)


In [ ]:
#Y_hat_all,best_roc_auc=train_data_grid (fps_train,fps_val,y_train)

In [ ]:
sub = pd.DataFrame(y_hat_all)
sub

,0,1,2,3,4,5,6,7,8,9,10
0,0.01,0.0500,0.005000,0.01,0.010000,0.00,0.060,0.01,0.050000,0.230000,0.000000
1,0.99,0.0040,0.020000,0.00,0.000000,0.00,0.000,0.00,0.020000,0.010000,0.006667
2,0.00,0.0100,0.000000,0.00,0.010000,0.00,0.130,0.00,0.210000,0.040000,0.000000
3,0.02,0.0100,0.026667,0.01,0.010000,0.00,0.010,0.00,0.000000,0.030000,0.005000
4,0.00,0.0100,0.356833,0.01,0.000000,0.00,0.065,0.00,0.343833,0.788786,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
5891,0.00,0.1800,0.105000,0.00,0.010000,0.01,0.100,0.00,0.118947,0.040000,0.010000
5892,0.03,0.0560,0.040000,0.00,0.003333,0.00,0.055,0.03,0.051667,0.185000,0.030000
5893,0.02,0.0450,0.092500,0.00,0.000000,0.00,0.035,0.00,0.040000,0.000000,0.000000
5894,0.90,0.0000,0.010000,0.02,0.010000,0.02,0.010,0.00,0.000000,0.020000,0.010000


#Final results

In [ ]:
sub = sub.rename(columns={0: 'task1', 1: 'task2', 2: 'task3', 2: 'task3', 3: 'task4', 4: 'task5', 5: 'task6', 6: 'task7', 7: 'task8', 8: 'task9', 9: 'task10', 10: 'task11'})
sub

,task1,task2,task3,task4,task5,task6,task7,task8,task9,task10,task11
0,0.01,0.0500,0.005000,0.01,0.010000,0.00,0.060,0.01,0.050000,0.230000,0.000000
1,0.99,0.0040,0.020000,0.00,0.000000,0.00,0.000,0.00,0.020000,0.010000,0.006667
2,0.00,0.0100,0.000000,0.00,0.010000,0.00,0.130,0.00,0.210000,0.040000,0.000000
3,0.02,0.0100,0.026667,0.01,0.010000,0.00,0.010,0.00,0.000000,0.030000,0.005000
4,0.00,0.0100,0.356833,0.01,0.000000,0.00,0.065,0.00,0.343833,0.788786,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
5891,0.00,0.1800,0.105000,0.00,0.010000,0.01,0.100,0.00,0.118947,0.040000,0.010000
5892,0.03,0.0560,0.040000,0.00,0.003333,0.00,0.055,0.03,0.051667,0.185000,0.030000
5893,0.02,0.0450,0.092500,0.00,0.000000,0.00,0.035,0.00,0.040000,0.000000,0.000000
5894,0.90,0.0000,0.010000,0.02,0.010000,0.02,0.010,0.00,0.000000,0.020000,0.010000


In [ ]:
sub.to_csv('output.csv', index=True)